In [ ]:
# Install dependencies
!pip install fastapi uvicorn transformers torch pyngrok

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import required libraries
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import torch
from torch import nn
from transformers import BertTokenizerFast, BertForTokenClassification
from pyngrok import ngrok
import uvicorn
import threading

In [ ]:
# Define FastAPI app
app = FastAPI()

# Pre-defined label mappings
ids_to_labels = {
    0: 'B-art', 1: 'B-eve', 2: 'B-geo', 3: 'B-gpe', 4: 'B-nat',
    5: 'B-org', 6: 'B-per', 7: 'B-tim', 8: 'I-art', 9: 'I-eve',
    10: 'I-geo', 11: 'I-gpe', 12: 'I-nat', 13: 'I-org', 14: 'I-per',
    15: 'I-tim', 16: 'O'
}

# Define model class
class BertModel(nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(ids_to_labels))

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, labels=labels, return_dict=False)
        return outputs

# Load tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
model = BertModel()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load model weights from Google Drive
model_path = '/content/drive/My Drive/mentoring/Kelas-NLP/models/ner-model.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

<ipython-input-39-fa182f8d624f>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias

In [ ]:
# Contoh kalimat
sentence = "Barack Obama was the 44th President of the United States."

# Inisialisasi tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# Fungsi evaluasi
def evaluate_one_text(model, sentence):
    text = tokenizer(sentence, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
    input_ids = text['input_ids']
    attention_mask = text['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs[0]
    predictions = torch.argmax(logits, dim=2)

    # Konversi ID prediksi ke label
    predicted_labels = [ids_to_labels[id] for id in predictions[0].tolist()]
    print(predicted_labels)

# Panggil fungsi evaluasi
evaluate_one_text(model, sentence)

['O', 'B-per', 'I-per', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'B-geo', 'I-geo', 'O', 'O', 'B-per', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'B-per', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'B-per', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'B-per', 'O', 'B-per', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [ ]:
# Pydantic schemas for request and response
class TextInput(BaseModel):
    text: str

class PredictionOutput(BaseModel):
    entities: List[str]  # Hanya berisi daftar label entitas

# Utility function for entity prediction
def predict_entities(text: str):
    # Tentukan perangkat (GPU jika ada, jika tidak CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenisasi input
    tokenized = tokenizer(text, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)

    # Gunakan model untuk mendapatkan output
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs[0]  # Ambil logits dari output
        predictions = torch.argmax(logits, dim=2)  # Ambil argmax di dimensi 2 untuk prediksi per token

    # Pastikan prediksi memiliki dimensi yang benar
    if predictions.dim() == 1:  # Jika hasilnya menjadi tensor 1D, tambahkan dimensi untuk 2D
        predictions = predictions.unsqueeze(0)  # Tambahkan dimensi tambahan di depan

    # Ambil word ids dari tokenized input
    word_ids = tokenized.word_ids()

    # Menghitung label menggunakan fungsi align_word_ids untuk filtering
    label_ids = align_word_ids(text)

    # Menyaring entitas dari prediksi dengan kata yang terdeteksi
    entities = []
    previous_word_idx = None
    for idx, word_idx in enumerate(word_ids):
        # Hanya ambil token yang bukan padding dan bukan subwords
        if word_idx is not None and word_idx != previous_word_idx:
            label_id = predictions[0][idx].item()  # Ambil nilai dari tensor 2D
            # Filter berdasarkan label_ids untuk memilih entitas yang valid
            if label_ids[idx] != -100 and ids_to_labels[label_id] != "O":  # "O" berarti bukan entitas
                word = tokenizer.convert_ids_to_tokens([input_ids[0][idx]])[0]  # Dapatkan kata asli dari token
                entity = ids_to_labels[label_id]  # Ambil entitas berdasarkan label_id
                entities.append(f"{word}: {entity}")  # Tambahkan entitas dan kata

        previous_word_idx = word_idx  # Update word_idx untuk langkah berikutnya

    return entities

In [ ]:
def align_word_ids(texts):
    # Tokenisasi input
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)  # -100 untuk padding atau subwords
        elif word_idx != previous_word_idx:
            label_ids.append(1)  # Misalnya label untuk entitas (ganti sesuai dengan kasus)
        else:
            label_ids.append(-100)  # -100 untuk subwords

        previous_word_idx = word_idx

    return label_ids

def evaluate_one_text(model, sentence):
    # Menentukan perangkat (GPU jika ada, jika tidak CPU)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    # Tokenisasi kalimat input
    text = tokenizer(sentence, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)

    # Menghitung label dari kalimat menggunakan fungsi align_word_ids
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    # Dapatkan output dari model
    with torch.no_grad():
        logits = model(input_id, mask)[0]  # Ambil logits output dari model

    # Pastikan logits memiliki dimensi yang benar untuk operasi indexing
    logits_clean = logits.squeeze(0)[label_ids.squeeze(0) != -100]  # Saring berdasarkan label_ids

    # Prediksi dengan argmax pada logits yang sudah bersih
    predictions = logits_clean.argmax(dim=1).tolist()  # Ambil argmax dari logits untuk prediksi

    # Mengkonversi ID prediksi menjadi label
    prediction_label = [ids_to_labels[i] for i in predictions]

    # Output hasil
    print(f"Sentence: {sentence}")
    print(f"Predictions: {prediction_label}")

In [ ]:
evaluate_one_text(model, 'Hello World Ikhwan')

Sentence: Hello World Ikhwan
Predictions: ['B-org', 'I-org', 'I-org']


In [ ]:
evaluate_one_text(model, 'Bill Gates is the founder of Microsoft')

Sentence: Bill Gates is the founder of Microsoft
Predictions: ['B-per', 'I-per', 'O', 'O', 'O', 'O', 'B-org']


In [ ]:
sentence = "Hello World Ikhwan"
entities = predict_entities(sentence)
print({"entities": entities})

{'entities': ['Hello: B-org', 'World: I-org', 'I: I-org']}


In [ ]:
sentence = "Bill Gates is the founder of Microsoft"
entities = predict_entities(sentence)
print({"entities": entities})

{'entities': ['Bill: B-per', 'Gates: I-per', 'Microsoft: B-org']}


In [ ]:
# FastAPI endpoints
@app.get("/")
def read_root():
    return {"message": "Welcome to the Named Entity Recognition API"}

@app.post("/predict", response_model=PredictionOutput)
def predict(text_input: TextInput):
    try:
        entities = predict_entities(text_input.text)
        return {"entities": entities}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
!ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Function to run FastAPI server
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start ngrok and FastAPI server
ngrok_tunnel = ngrok.connect(8000)
print(f"Public URL: {ngrok_tunnel.public_url}")

# Run FastAPI in a thread
thread = threading.Thread(target=run_app)
thread.start()

Public URL: https://4490-35-236-210-244.ngrok-free.app


In [ ]:
import requests

url = "https://4490-35-236-210-244.ngrok-free.app/predict"
data = {"text": "Bill Gates is the founder of Microsoft"}
response = requests.post(url, json=data)

print(response.json())

INFO:     35.236.210.244:0 - "POST /predict HTTP/1.1" 200 OK
{'entities': ['Bill: B-per', 'Gates: I-per', 'Microsoft: B-org']}


In [ ]:
url = "https://4490-35-236-210-244.ngrok-free.app/predict"
data = {"text": "Hello World Ikhwan"}
response = requests.post(url, json=data)

print(response.json())

INFO:     35.236.210.244:0 - "POST /predict HTTP/1.1" 200 OK
{'entities': ['Hello: B-org', 'World: I-org', 'I: I-org']}
